## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.csv("/user/sprenner/lookup_mai.csv", sep="\t")

In [5]:
df_lookup.count()

831654692

In [6]:
#df_lookup.limit(10).toPandas()

In [7]:
df_lookup_renamed = df_lookup.toDF('SCOPE', 'NAME', 'CHILD_SCOPE', 'CHILD_NAME', 'DID_TYPE', 'CHILD_TYPE', 
                                   'LENGTH', 'BYTES', 'ADLER32', 'MD5', 'RULE_EVALUATION', 'UPDATED_AT', 
                                   'CREATED_AT', 'EVENTS')

In [9]:
df_mai = spark.read.json("/user/sprenner/training_official_mai_hourly_cleaned_datasets.json")

In [10]:
df_mai = df_mai.withColumn("join_key_mai", concat_ws(':', df_mai["datasetScope"], df_mai["dataset"]))
df_mai.createOrReplaceTempView("mai")
df_lookup_renamed = df_lookup_renamed.withColumn("join_key_lookup", concat_ws(':', df_lookup_renamed["SCOPE"], df_lookup_renamed["NAME"]))
df_lookup_renamed.createOrReplaceTempView("lookup_renamed")

In [11]:
spark.sql("SELECT * FROM lookup_renamed WHERE join_key_lookup IS NULL").toPandas()

Empty DataFrame
Columns: [SCOPE, NAME, CHILD_SCOPE, CHILD_NAME, DID_TYPE, CHILD_TYPE, LENGTH, BYTES, ADLER32, MD5, RULE_EVALUATION, UPDATED_AT, CREATED_AT, EVENTS, join_key_lookup]
Index: []

In [12]:
df_lookup_reduced = spark.sql("SELECT *\
                                 FROM lookup_renamed\
                                 WHERE SCOPE != 'panda' AND DID_TYPE == 'D' AND (NAME LIKE 'data%' OR NAME LIKE 'mc%')")
df_lookup_reduced.count()

624370615

In [13]:
df_mai.limit(5).toPandas()

account  accountHash         clientState  clientStateHash  \
0  lchristo  1908036066   DONE               -878527175         
1  pswiersk  2132699417   DONE               -878527175         
2  pswiersk  2132699417   DONE               -878527175         
3  ysano     1418469109   DONE               -878527175         
4  ysano     1418469109   ServiceUnavailable -1394468120        

                                                                                                                              dataset  \
0  mc16_13TeV.364122.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV140_280_BFilter.deriv.DAOD_EGAM7.e5299_s3126_r9364_r9315_p3469_tid13424759_00   
1  data17_13TeV.00339070.physics_EnhancedBias.merge.AOD.r10365_r10366_p3486_tid13518449_00                                              
2  data17_13TeV.00339070.physics_EnhancedBias.merge.AOD.r10365_r10366_p3486_tid13518449_00                                              
3  data17_13TeV.00337705.physics_Main.deriv.DAOD_EXOT8.f877_m1885_p3481_tid13457757_00                                                  
4  data17_13TeV.00337705.physics_Main.deriv.DAOD_EXOT8.f877_m1885_p3481_tid13457757_00                                                  

   datasetHash  datasetScope  datasetScopeHash  \
0 -1541402103   mc16_13TeV   -1447489496         
1  1673599491   data17_13TeV -841583270          
2  1673599491   data17_13TeV -841583270          
3  1435987549   data17_13TeV -841583270          
4  1435987549   data17_13TeV -841583270          

                                dataset_version  dataset_versionHash  \
0  e5299_s3126_r9364_r9315_p3469_tid13424759_00  1508153648            
1  r10365_r10366_p3486_tid13518449_00            306242405             
2  r10365_r10366_p3486_tid13518449_00            306242405             
3  f877_m1885_p3481_tid13457757_00               540117808             
4  f877_m1885_p3481_tid13457757_00               540117808             

     datatype  datatypeHash eventType  eventTypeHash eventVersion  \
0  DAOD_EGAM7  1921601987    download  59378080       1.14.11       
1  AOD         1206945951    download  59378080       1.14.11       
2  AOD         1206945951    download  59378080       1.14.11       
3  DAOD_EXOT8 -754389316     download  59378080       1.14.11       
4  DAOD_EXOT8 -754389316     download  59378080       1.14.11       

   eventVersionHash     fileScope                                 filename  \
0 -2044360925        mc16_13TeV    DAOD_EGAM7.13424759._000153.pool.root.1   
1 -2044360925        data17_13TeV  AOD.13518449._000050.pool.root.1          
2 -2044360925        data17_13TeV  AOD.13518449._000054.pool.root.1          
3 -2044360925        data17_13TeV  DAOD_EXOT8.13457757._000274.pool.root.1   
4 -2044360925        data17_13TeV  DAOD_EXOT8.13457757._000278.pool.root.1   

   filenameHash  filesize_double  hits           hostname  hostnameHash  \
0  1337691228    3.717195e+09     1     lxplus038.cern.ch  213814853      
1 -1849017532    1.117869e+09     2     lxplus056.cern.ch -1117453067     
2  820198977     9.156416e+08     1     lxplus056.cern.ch -1117453067     
3 -1083173376    1.338823e+09     5     lxatut10.cern.ch   492469658      
4 -362019174     1.473129e+09     2     lxatut10.cern.ch   492469658      

            hour   hourHash prod_step  prod_stepHash       project  \
0  2018-05-14 11  751511098  deriv    -1058503065     mc16_13TeV     
1  2018-05-14 11  751511098  merge     874345773      data17_13TeV   
2  2018-05-14 11  751511098  merge     874345773      data17_13TeV   
3  2018-05-14 11  751511098  deriv    -1058503065     data17_13TeV   
4  2018-05-14 11  751511098  deriv    -1058503065     data17_13TeV   

   projectHash protocol  protocolHash           remoteSite  remoteSiteHash  \
0 -1447489496   gsiftp   2128034789    SLACXRD_DATADISK     1322032958       
1 -841583270    gsiftp   2128034789    CERN-PROD_DATADISK  -2057286004       
2 -841583270    srm      863299160     TOKYO-LCG2_DATADISK -211313

In [14]:
df_lookup_reduced.createOrReplaceTempView('lookup_reduced')
spark.sql("SELECT * FROM lookup_reduced WHERE join_key_lookup == 'data15_13TeV:data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00'").toPandas()
#spark.sql("SELECT * FROM lookup_reduced WHERE join_key_lookup IS NULL").toPandas()

SCOPE  \
0    data15_13TeV   
1    data15_13TeV   
2    data15_13TeV   
3    data15_13TeV   
4    data15_13TeV   
5    data15_13TeV   
6    data15_13TeV   
7    data15_13TeV   
8    data15_13TeV   
9    data15_13TeV   
10   data15_13TeV   
11   data15_13TeV   
12   data15_13TeV   
13   data15_13TeV   
14   data15_13TeV   
15   data15_13TeV   
16   data15_13TeV   
17   data15_13TeV   
18   data15_13TeV   
19   data15_13TeV   
20   data15_13TeV   
21   data15_13TeV   
22   data15_13TeV   
23   data15_13TeV   
24   data15_13TeV   
25   data15_13TeV   
26   data15_13TeV   
27   data15_13TeV   
28   data15_13TeV   
29   data15_13TeV   
..            ...   
97   data15_13TeV   
98   data15_13TeV   
99   data15_13TeV   
100  data15_13TeV   
101  data15_13TeV   
102  data15_13TeV   
103  data15_13TeV   
104  data15_13TeV   
105  data15_13TeV   
106  data15_13TeV   
107  data15_13TeV   
108  data15_13TeV   
109  data15_13TeV   
110  data15_13TeV   
111  data15_13TeV   
112  data15_13TeV   
113  data15_13TeV   
114  data15_13TeV   
115  data15_13TeV   
116  data15_13TeV   
117  data15_13TeV   
118  data15_13TeV   
119  data15_13TeV   
120  data15_13TeV   
121  data15_13TeV   
122  data15_13TeV   
123  data15_13TeV   
124  data15_13TeV   
125  data15_13TeV   
126  data15_13TeV   

                                                                                      NAME  \
0    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
1    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
2    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
3    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
4    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
5    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
6    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
7    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
8    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
9    data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
10   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
11   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
12   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
13   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
14   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
15   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
16   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
17   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
18   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
19   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
20   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
21   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
22   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
23   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
24   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
25   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
26   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
27   data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_t

In [15]:
df_final = df_mai.join(df_lookup_reduced, df_mai.join_key_mai == df_lookup_reduced.join_key_lookup, 'left_outer')

In [16]:
df_final.count()

909789389

In [17]:
df_final.createOrReplaceTempView("final")
spark.sql("SELECT * FROM final WHERE join_key_lookup IS NULL").limit(10).toPandas()

account  accountHash clientState  clientStateHash  \
0  bejaeger -479767920    DONE       -878527175         
1  bejaeger -479767920    DONE       -878527175         
2  bejaeger -479767920    DONE       -878527175         
3  bejaeger -479767920    DONE       -878527175         
4  bejaeger -479767920    DONE       -878527175         
5  bejaeger -479767920    DONE       -878527175         
6  bejaeger -479767920    DONE       -878527175         
7  bejaeger -479767920    DONE       -878527175         
8  bejaeger -479767920    DONE       -878527175         
9  bejaeger -479767920    DONE       -878527175         

                                                                 dataset  \
0  data16_13TeV.00309640.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518   
1  data16_13TeV.00309640.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518   
2  data16_13TeV.00309640.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518   
3  data16_13TeV.00309640.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518   
4  data16_13TeV.00309640.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518   
5  data16_13TeV.00309640.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518   
6  data16_13TeV.00309640.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518   
7  data16_13TeV.00309640.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518   
8  data16_13TeV.00309640.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518   
9  data16_13TeV.00309640.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518   

   datasetHash  datasetScope  datasetScopeHash    dataset_version  \
0  22268422     data16_13TeV -915656957         r9264_p3083_p3518   
1  22268422     data16_13TeV -915656957         r9264_p3083_p3518   
2  22268422     data16_13TeV -915656957         r9264_p3083_p3518   
3  22268422     data16_13TeV -915656957         r9264_p3083_p3518   
4  22268422     data16_13TeV -915656957         r9264_p3083_p3518   
5  22268422     data16_13TeV -915656957         r9264_p3083_p3518   
6  22268422     data16_13TeV -915656957         r9264_p3083_p3518   
7  22268422     data16_13TeV -915656957         r9264_p3083_p3518   
8  22268422     data16_13TeV -915656957         r9264_p3083_p3518   
9  22268422     data16_13TeV -915656957         r9264_p3083_p3518   

   dataset_versionHash    datatype  datatypeHash eventType  eventTypeHash  \
0  568651403            DAOD_JETM5  414887654     download  59378080        
1  568651403            DAOD_JETM5  414887654     download  59378080        
2  568651403            DAOD_JETM5  414887654     download  59378080        
3  568651403            DAOD_JETM5  414887654     download  59378080        
4  568651403            DAOD_JETM5  414887654     download  59378080        
5  568651403            DAOD_JETM5  414887654     download  59378080        
6  568651403            DAOD_JETM5  414887654     download  59378080        
7  568651403            DAOD_JETM5  414887654     download  59378080        
8  568651403            DAOD_JETM5  414887654     download  59378080        
9  568651403            DAOD_JETM5  414887654     download  59378080        

  eventVersion  eventVersionHash     fileScope  \
0  1.14.11     -2044360925        data16_13TeV   
1  1.14.11     -2044360925        data16_13TeV   
2  1.14.11     -2044360925        data16_13TeV   
3  1.14.11     -2044360925        data16_13TeV   
4  1.14.11     -2044360925        data16_13TeV   
5  1.14.11     -2044360925        data16_13TeV   
6  1.14.11     -2044360925        data16_13TeV   
7  1.14.11     -2044360925        data16_13TeV   
8  1.14.11     -2044360925        data16_13TeV   
9  1.14.11     -2044360925        data16_13TeV   

                                  filename  filenameHash  filesize_double  \
0  DAOD_JETM5.13864527._000139.pool.root.1  837572028     605680.0          
1  DAOD_JETM5.13864527._000140.pool.root.1  1697050277    1636524.0         
2  DAOD_JETM5.13864527._000141.pool.root.1  1342708849    2278841.0         
3  DAOD_JETM5.13864527._000142.pool.root.1 -1922525630    2742615.0     

In [18]:
df_final_dropped = df_final.select('join_key_lookup','CHILD_SCOPE','CHILD_NAME','BYTES')

In [19]:
df_final_dropped.printSchema()

root
 |-- join_key_lookup: string (nullable = true)
 |-- CHILD_SCOPE: string (nullable = true)
 |-- CHILD_NAME: string (nullable = true)
 |-- BYTES: string (nullable = true)



In [20]:
df_final_dropped = df_final_dropped.dropDuplicates()

In [21]:
df_final_dropped.count()

4492386

In [22]:
df_final_dropped.createOrReplaceTempView("final_dropped")
spark.sql("SELECT COUNT(*) FROM final_dropped WHERE join_key_lookup IS NULL").toPandas()
#vermutlich bleibt nur ein eintrag über, weil alle anderen durch das selektive dropduplicates entfernt wurden

count(1)
0  1

In [23]:
df_final_dropped = df_final_dropped.dropna(subset=['join_key_lookup'])

In [24]:
df_final_dropped.count()

4492385

In [25]:
df_final_dropped_json = df_final_dropped.toJSON()
df_final_dropped_json.first()

u'{"join_key_lookup":"data15_13TeV:data15_13TeV.00276952.physics_Main.deriv.DAOD_HIGG3D1.r9264_p3083_p3388_tid12645225_00","CHILD_SCOPE":"data15_13TeV","CHILD_NAME":"DAOD_HIGG3D1.12645225._000007.pool.root.1","BYTES":"1282726231"}'

In [32]:
df_final_dropped.write.format("json").save("/user/sprenner/lookup_final_mai_small.json")

In [27]:
df_final_dropped_count = df_final_dropped.groupBy('join_key_lookup').count()

In [33]:
df_final_dropped_count.write.format("json").save("/user/sprenner/lookup_final_mai_count.json")

In [29]:
df_final_dropped_count.orderBy('count', ascending=False).toPandas()
#12362 rows mit null drin, sollte eine weniger sein

join_key_lookup  \
0      mc15_13TeV:mc15_13TeV.364250.Sherpa_222_NNPDF30NNLO_llll.evgen.log.e5894_tid11041837_00                                                                                
1      mc15_13TeV:mc15_13TeV.364253.Sherpa_222_NNPDF30NNLO_lllv.evgen.log.e5916_tid11839576_00                                                                                
2      mc15_13TeV:mc15_13TeV.364250.Sherpa_222_NNPDF30NNLO_llll.evgen.EVNT.e5894_tid11041837_00                                                                               
3      mc15_13TeV:mc15_13TeV.364153.Sherpa_221_NNPDF30NNLO_Znunu_MAXHTPTV280_500_BFilter.evgen.EVNT.e5308_tid09300448_00                                                      
4      mc15_13TeV:mc15_13TeV.364253.Sherpa_222_NNPDF30NNLO_lllv.evgen.log.e5916_tid13551044_00                                                                                
5      mc15_13TeV:mc15_13TeV.364253.Sherpa_222_NNPDF30NNLO_lllv.evgen.log.e5916_tid11125422_00                                                                                
6      mc15_13TeV:mc15_13TeV.366002.Sherpa_221_NNPDF30NNLO_Znunu_PTV100_140_MJJ0_500.evgen.EVNT.e6670_tid13917819_00                                                          
7      mc15_13TeV:mc15_13TeV.366001.Sherpa_221_NNPDF30NNLO_Znunu_PTV70_100.evgen.EVNT.e6670_tid13917808_00                                                                    
8      mc15_13TeV:mc15_13TeV.361224.Epos_minbias_inelastic.evgen.EVNT.e3908_tid08736240_00                                                                                    
9      mc15_13TeV:mc15_13TeV.366005.Sherpa_221_NNPDF30NNLO_Znunu_PTV140_280_MJJ0_500.evgen.EVNT.e6670_tid13917850_00                                                          
10     mc15_13TeV:mc15_13TeV.345054.PowhegPythia8EvtGen_NNPDF3_AZNLO_WpH125J_MINLO_lvbb_VpT.evgen.TXT.e5706_tid13720333_00                                                    
11     mc16_13TeV:mc16_13TeV.361107.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zmumu.simul.HITS.e3601_s3126_tid11363382_00                                                              
12     mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.evgen.EVNT.e3601_tid13468331_00                                                                      
13     mc15_13TeV:mc15_13TeV.345054.PowhegPythia8EvtGen_NNPDF3_AZNLO_WpH125J_MINLO_lvbb_VpT.evgen.TXT.e5706_tid12017482_00                                                    
14     mc16_13TeV:mc16_13TeV.361107.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zmumu.simul.HITS.e3601_e5984_s3126_tid13490040_00                                                        
15     mc15_13TeV:mc15_13TeV.364122.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV140_280_BFilter.evgen.EVNT.e5299_tid09741824_00                                                        
16     mc16_13TeV:mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.simul.HITS.e6337_e5984_s3126_tid12860054_00                                                       
17     mc16_13TeV:mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.simul.HITS.e6337_e5984_s3126_tid13453091_00                                                       
18     mc16_13TeV:mc16_13TeV.361107.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zmumu.simul.HITS.e3601_s3126_tid11012007_00                                                              
19     mc15_13TeV:mc15_13TeV.345054.PowhegPythia8EvtGen_NNPDF3_AZNLO_WpH125J_MINLO_lvbb_VpT.evgen.TXT.e5706_tid10406367_00                                                    
20     mc15_13TeV:mc15_13TeV.364139.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV280_500_BFilter.evgen.EVNT.e5313_tid09307168_00                                                    
21     mc15_13TeV:mc15_13TeV.345054.PowhegPythia8EvtGen_NNPDF3_AZNLO_WpH125J_MINLO_lvbb_VpT.evgen.TXT.e5706_tid10412317_00                                                    
22     mc15_13TeV:mc15_13TeV.364125.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV280_500_BFilter.evgen.EVNT.e5299_tid09265966_00             

In [30]:
df_final_dropped.createOrReplaceTempView("dropped")
df_count = spark.sql("SELECT join_key_lookup, COUNT(DISTINCT *) AS count FROM dropped GROUP BY join_key_lookup ORDER BY count DESC")
df_count.toPandas()
##hier war ganz unten als jetzte die Gruppe "None" mit count 0

join_key_lookup  \
0      mc15_13TeV:mc15_13TeV.364250.Sherpa_222_NNPDF30NNLO_llll.evgen.log.e5894_tid11041837_00                                                                               
1      mc15_13TeV:mc15_13TeV.364253.Sherpa_222_NNPDF30NNLO_lllv.evgen.log.e5916_tid11839576_00                                                                               
2      mc15_13TeV:mc15_13TeV.364250.Sherpa_222_NNPDF30NNLO_llll.evgen.EVNT.e5894_tid11041837_00                                                                              
3      mc15_13TeV:mc15_13TeV.364153.Sherpa_221_NNPDF30NNLO_Znunu_MAXHTPTV280_500_BFilter.evgen.EVNT.e5308_tid09300448_00                                                     
4      mc15_13TeV:mc15_13TeV.364253.Sherpa_222_NNPDF30NNLO_lllv.evgen.log.e5916_tid13551044_00                                                                               
5      mc15_13TeV:mc15_13TeV.364253.Sherpa_222_NNPDF30NNLO_lllv.evgen.log.e5916_tid11125422_00                                                                               
6      mc15_13TeV:mc15_13TeV.366002.Sherpa_221_NNPDF30NNLO_Znunu_PTV100_140_MJJ0_500.evgen.EVNT.e6670_tid13917819_00                                                         
7      mc15_13TeV:mc15_13TeV.366001.Sherpa_221_NNPDF30NNLO_Znunu_PTV70_100.evgen.EVNT.e6670_tid13917808_00                                                                   
8      mc15_13TeV:mc15_13TeV.361224.Epos_minbias_inelastic.evgen.EVNT.e3908_tid08736240_00                                                                                   
9      mc15_13TeV:mc15_13TeV.366005.Sherpa_221_NNPDF30NNLO_Znunu_PTV140_280_MJJ0_500.evgen.EVNT.e6670_tid13917850_00                                                         
10     mc15_13TeV:mc15_13TeV.345054.PowhegPythia8EvtGen_NNPDF3_AZNLO_WpH125J_MINLO_lvbb_VpT.evgen.TXT.e5706_tid13720333_00                                                   
11     mc16_13TeV:mc16_13TeV.361107.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zmumu.simul.HITS.e3601_s3126_tid11363382_00                                                             
12     mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.evgen.EVNT.e3601_tid13468331_00                                                                     
13     mc15_13TeV:mc15_13TeV.345054.PowhegPythia8EvtGen_NNPDF3_AZNLO_WpH125J_MINLO_lvbb_VpT.evgen.TXT.e5706_tid12017482_00                                                   
14     mc16_13TeV:mc16_13TeV.361107.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zmumu.simul.HITS.e3601_e5984_s3126_tid13490040_00                                                       
15     mc15_13TeV:mc15_13TeV.364122.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV140_280_BFilter.evgen.EVNT.e5299_tid09741824_00                                                       
16     mc16_13TeV:mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.simul.HITS.e6337_e5984_s3126_tid12860054_00                                                      
17     mc16_13TeV:mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.simul.HITS.e6337_e5984_s3126_tid13453091_00                                                      
18     mc16_13TeV:mc16_13TeV.361107.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zmumu.simul.HITS.e3601_s3126_tid11012007_00                                                             
19     mc15_13TeV:mc15_13TeV.364139.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV280_500_BFilter.evgen.EVNT.e5313_tid09307168_00                                                   
20     mc15_13TeV:mc15_13TeV.345054.PowhegPythia8EvtGen_NNPDF3_AZNLO_WpH125J_MINLO_lvbb_VpT.evgen.TXT.e5706_tid10406367_00                                                   
21     mc15_13TeV:mc15_13TeV.345054.PowhegPythia8EvtGen_NNPDF3_AZNLO_WpH125J_MINLO_lvbb_VpT.evgen.TXT.e5706_tid10412317_00                                                   
22     mc15_13TeV:mc15_13TeV.364125.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV280_500_BFilter.evgen.EVNT.e5299_tid09265966_00                                   

In [31]:
df_final_dropped_count.createOrReplaceTempView("counts")
spark.sql("SELECT * FROM counts WHERE join_key_lookup == 'None' OR join_key_lookup IS NULL").toPandas()
#hier war ein eintrag, "None" mit count 1

Empty DataFrame
Columns: [join_key_lookup, count]
Index: []